In [ ]:
%pylab notebook

import pandas as pd
import numpy as np
import datetime
import os

import ipywidgets as widgets

from cognite.client import CogniteClient   

In [ ]:
client = CogniteClient(api_key=os.environ["OID_API_KEY"]) 

### Get all timeseries relevant for the compressor

In [ ]:
# If we know the tag
compressor_tag = '23-KA-9101'
client.assets.get_assets(name=compressor_tag).to_pandas()

In [ ]:
time_series = client.time_series.get_time_series(asset_id=3047932288982463).to_pandas()
time_series

In [ ]:
# Let us check how much data we have
start = 0
end = 'now'
granularity = '1d'
aggregates = ['avg']
data = client.datapoints.get_datapoints_frame(time_series = list(time_series.name.values), start=start, end=end, aggregates=aggregates, granularity=granularity)

In [ ]:
df = data.copy()
df.timestamp = pd.to_datetime(df.timestamp, unit='ms')
df

In [ ]:
data.describe()

In [ ]:
df_nan = 0.5*(~df.isnull().astype(int)+1)

plt.figure(figsize=(10,5))
A= plt.pcolor(df_nan.drop(['timestamp'], axis=1).values.transpose())
yloc = np.arange(0.5, len(df_nan.columns)-1+0.5)
ytick = [w[0] for w in list(df_nan.drop(['timestamp'],axis=1).columns.str.split(':'))]
A =plt.yticks(yloc, ytick, fontsize='xx-small')
xtick = [str(w).split('-')[0] for w in df.timestamp[df.index%365==0].values] 
xloc = np.arange(0, len(df), 365)
A = plt.xticks(xloc, xtick)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
T = pd.to_datetime(data.timestamp, unit='ms')
ax.plot(T, data['VAL_23-KA-9101_ASP:VALUE|average'], label='Power kW')
ax2 = ax.twinx()
ax2.plot(T, data['VAL_23-KA-9101_APR:VALUE|average'], color='red', label='p/p')
ax.legend()
ax2.legend()

In [ ]:
#Something seems to have happend between 2nd and 30th of June 2016
min_start_time = int(datetime.datetime(2016, 6, 1).timestamp()*1000)
max_end_time   = int(datetime.datetime(2016, 8, 15).timestamp()*1000)
events = client.events.search_for_events( min_start_time=min_start_time, max_end_time=max_end_time).to_pandas()
compressor_events = events[[3047932288982463 in aid for aid in events.assetIds]]
compressor_events

In [ ]:
DE = compressor_events[['WORKORDER_NUMBER', 'WORKORDER_SHUTDOWNDESC']].copy()
DE['startTime'] = pd.to_datetime(compressor_events.startTime, unit='ms')
DE['endTime'] = pd.to_datetime(compressor_events.endTime, unit='ms')
DE.sort_values(by='startTime', inplace=True)
DE

In [ ]:
scond = ['shutdown' in str(w) for w in DE['WORKORDER_SHUTDOWNDESC']]
shutdown = DE[scond].reset_index(drop=True)
shutdown

In [ ]:
plt.plot(shutdown['startTime'], np.ones(len(shutdown)), 'D', color='purple')

In [ ]:
# Let us refine the search
start = datetime.datetime(2016, 5, 20)
end   = datetime.datetime(2016, 8, 1)
granularity = '1m'
aggregates = ['avg']
data = client.datapoints.get_datapoints_frame(time_series=['VAL_23-KA-9101_ASP:VALUE', 'VAL_23-KA-9101_APR:VALUE'], 
                            start = start, 
                            end = end, 
                            granularity = granularity, 
                            aggregates = aggregates)


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
T = pd.to_datetime(data.timestamp, unit='ms')
ax.plot(T, data['VAL_23-KA-9101_ASP:VALUE|average'], label='Power kW')
ax2 = ax.twinx()
ax2.plot(T, data['VAL_23-KA-9101_APR:VALUE|average'], color='red', label='p/p')
ax.legend()
ax2.legend()

In [ ]:
plt.plot(shutdown['startTime'], np.ones(len(shutdown)), 'D', color='purple')

In [ ]:
# What is up with the oscilations
# Let us refine the search
start = datetime.datetime(2016, 5, 20)
end   = datetime.datetime(2016, 5, 21)
granularity = '1s'
aggregates = ['avg']
data = client.datapoints.get_datapoints_frame(time_series=['VAL_23-KA-9101_ASP:VALUE', 'VAL_23-KA-9101_APR:VALUE'], 
                            start = start, 
                            end = end, 
                            granularity = granularity, 
                            aggregates = aggregates)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
T = pd.to_datetime(data.timestamp, unit='ms')
ax.plot(T, data['VAL_23-KA-9101_ASP:VALUE|average'], label='Power kW')
ax2 = ax.twinx()
ax2.plot(T, data['VAL_23-KA-9101_APR:VALUE|average'], color='red', label='p/p')
ax.legend()
ax2.legend()

In [ ]:
# What is up with the oscilations
# Let us refine the search
start = 0
end   = 'now'
granularity = '1d'
aggregates = ['avg','tv']
data = client.datapoints.get_datapoints_frame(time_series=['VAL_23-KA-9101_ASP:VALUE', 'VAL_23-KA-9101_APR:VALUE'], 
                            start = start, 
                            end = end, 
                            granularity = granularity, 
                            aggregates = aggregates)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
T = pd.to_datetime(data.timestamp, unit='ms')
ax.plot(T, data['VAL_23-KA-9101_APR:VALUE|totalvariation'], label='Power kW')
ax2 = ax.twinx()
ax2.plot(T, data['VAL_23-KA-9101_APR:VALUE|average'], color='red', label='p/p')
ax.legend()
ax2.legend()